In [83]:
from qiskit.circuit import QuantumCircuit, Parameter
import numpy as np
phi = Parameter("1")
phi2 = Parameter("2")
qr = QuantumRegister(nr_of_qubits)
circuit = QuantumCircuit(qr)
circuit.h([0])
circuit.h([1])
circuit.h([2])
circuit.h([3])
circuit.rx(phi,qubit=[0])
circuit.rx(phi2,qubit=[1])
print(circuit)
print(circuit.parameters)
value = np.random.random()
parameters = [value for _ in range(2)]
print(circuit.bind_parameters(parameters))

       ┌───┐┌───────┐
q68_0: ┤ H ├┤ Rx(1) ├
       ├───┤├───────┤
q68_1: ┤ H ├┤ Rx(2) ├
       ├───┤└───────┘
q68_2: ┤ H ├─────────
       ├───┤         
q68_3: ┤ H ├─────────
       └───┘         
ParameterView([Parameter(1), Parameter(2)])
       ┌───┐┌───────────────────────┐
q68_0: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤├───────────────────────┤
q68_1: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤└───────────────────────┘
q68_2: ┤ H ├─────────────────────────
       ├───┤                         
q68_3: ┤ H ├─────────────────────────
       └───┘                         


In [23]:
individual = "0000000"
elitism_population = ["1110111","1111111"]
match_percentage = []
def match(subject_1,subject_2):
    return sum([int(subject_1[i] == subject_2[i]) for i in range(0,len(subject_1))])/len(subject_1)
for elite in elitism_population:
    match_percentage.append(match(individual,elite))
    print(match_percentage)
print(sum(match_percentage)/len(match_percentage))

[0.14285714285714285]
[0.14285714285714285, 0.0]
0.07142857142857142


In [13]:
from experiments.Experiments import saveLoad
from CLQGA import Individual
output = saveLoad("load","experiment_9_april_MUC__0", None)
experiment_population = output[0]
experiment_duration = output[1]
experiment_average_fitness_50_increment = output[2]
experiment_average_crowd_score_50_increment = output[3]

In [14]:
# [population,time.gmtime((time.perf_counter() - start_total_run_time)),data_average_fitness, data_average_crowd_score]
print(output[1:])


[time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=20, tm_min=37, tm_sec=57, tm_wday=3, tm_yday=1, tm_isdst=0), [51364.6847869238, 49123.942950914185, 40966.57192727442, 47322.143118011205, 47601.98114940698, 47502.46705865293, 47771.330639944164, 45220.76456463997, 43942.712421448814, 50280.511427321006, 44020.73849075216, 47864.17897823599, 54085.52749314641, 45037.50666593698, 42338.821986508454, 45433.28073498909, 44444.96480977509, 46810.42378862526, 40467.290007518124, 42800.14815031977, 53543.78962664479, 54167.18295290197, 46261.843609307245, 52142.33406230515, 45241.986173349156, 42609.12716555319, 43864.976990534466, 42351.37234071031, 54460.315892682804, 47578.673909670244, 46386.31521717222, 48774.83928691846, 47695.26848224773, 48094.25362135701, 49126.757879732184, 51112.137260485106, 46799.11006353039, 49429.54725129526, 49438.23014243184, 50331.39936807622], [0.4311515151515152, 0.3923636363636364, 0.42521212121212115, 0.4148181818181817, 0.4102424242424242, 0